In [2]:
import numpy as np
import pandas as pd
import requests
#The SKOS Graph management libraries
import rdflib
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import RDF, SKOS
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel

In [31]:
mpdata = mpd()
df_tool_flat =mpdata.getMPItems ("toolsandservices", True)
df_publication_flat =mpdata.getMPItems ("publications", True)
df_trainingmaterials_flat =mpdata.getMPItems ("trainingmaterials", True)
df_workflows_flat =mpdata.getMPItems ("workflows", True)
df_datasets_flat =mpdata.getMPItems ("datasets", True)

getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...


#### Get the dataframe containing items and their properties

In [15]:
utils=hel.Util()
resultfields=['persistentId', 'MPUrl', 'category', 'label', 'type.code', 'type.label', 'concept.code', 'concept.label', 'concept.uri', 'concept.vocabulary.scheme']
udf_alprop=utils.getAllPropertiesBySources()
udf_alprop=udf_alprop.loc[ : ,resultfields]

#### Get the keywords vocabulary as RDF graph

The following cell creates an RDF Graph containing the [vocabulary of the SSHOC keywords](https://vocabs-downloads.acdh.oeaw.ac.at/vocabs-common/SSHOpenMarketplace/sshomp-keyword/sshomp-keyword.ttl), downloaded in ttl format.

In [32]:
#create an RDF Graph
keywords_rdf = rdflib.Graph() 
#insert in the graph the SKOS vocabulary file obtained from the SSH Vocabulary Commons server
keywords_rdf.parse("https://vocabs-downloads.acdh.oeaw.ac.at/vocabs-common/SSHOpenMarketplace/sshomp-keyword/sshomp-keyword.ttl", format="turtle")

<Graph identifier=Ne002729aa55f41be90d47dad08729d06 (<class 'rdflib.graph.Graph'>)>

Test if the voabulary is correctly downloaded by checking if two keywords are present in it

In [33]:
k = URIRef("https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/mushrooms")
if (k, RDF.type, SKOS.Concept) in keywords_rdf:
    print(f"This graph knows that {k} is a Concept!")
kl=Literal('media', lang='en')
if (None, SKOS.prefLabel, kl) in keywords_rdf:
    print(f"This graph knows that {kl} is a preferred label for a Concept!")

This graph knows that https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/mushrooms is a Concept!
This graph knows that media is a preferred label for a Concept!


Filter the dataframe of items selecting only keywords

In [34]:
itemswkws=udf_alprop[udf_alprop['type.label']=='Keyword']
#itemswkws.head()

#### Navigate the RDF Graph and for every Concept print if it has a 'label', a 'notation' or neither

In [35]:
for s, p, o in keywords_rdf.triples((None, RDF.type, SKOS.Concept)):
    if (s, SKOS.prefLabel, None) in keywords_rdf:
        print(f"The Concept {s} has a label property, value is:")
        print(keywords_rdf.value(s, SKOS.prefLabel, None))
        continue;
    if (s, SKOS.notation, None) in keywords_rdf:
        print(f"The Concept {s} has a notation property, value is:")
        print(keywords_rdf.value(s, SKOS.notation, None))
        continue
    else:
        print(f"The Concept: \n{s}\ndoes not have label or notation")

The Concept https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/1-grams has a label property, value is:
1-grams
The Concept https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/18th-century has a label property, value is:
18th-century
The Concept https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/18th-century-literature has a label property, value is:
18th-century-literature
The Concept https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/19th-century has a label property, value is:
19th-century
The Concept https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/19th-century-literature has a label property, value is:
19th-century-literature
The Concept https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/3d has a label property, value is:
3D
The Concept https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/3d-graphics has a label property, value is:
3d-graphics
The Concept https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/abstracts has 

#### Iterate on the dataframe containing keywords to check if they have a Vocabulary entry, create a dataframe of keywords not having entries

In [36]:
itemsnokw=pd.DataFrame()
for n, row in itemswkws.iterrows():
    pk=row['concept.label']
    if ('http' in pk):
        continue
    kl=Literal(pk, lang='en')
    kn=Literal(pk)
    if (None, SKOS.prefLabel, kl) in keywords_rdf:
        continue
    if (None, SKOS.notation, kn) in keywords_rdf:
#         s=keywords_rdf.value(None, SKOS.notation, kn)
#         print(s)
#         sub=URIRef(s)
#         myLabel=Literal(kn, lang='en')
#         keywords_rdf.add((sub, SKOS.prefLabel, myLabel))
        continue
          
    print (f"Warning {pk} is a Concept URI not having label or notation")
#     pku=pk.replace(" ","+")
#     ku=URIRef(f'https://vocabs.sshopencloud.eu/vocabularies/sshomp-keyword/{pku}')
#     myLabel=Literal(pk.replace('+', ' '), lang='en')
#     keywords_rdf.add((ku, SKOS.prefLabel, myLabel))
    muit={}
    muit['MPUrl']=row.MPUrl
    muit['label']=row.label
    muit['code']=row['concept.code']
    muit['code.uri']=row['concept.uri']
    itemsnokw=itemsnokw.append(muit,  ignore_index=True)

Warning DARIAH National Resource is a Concept URI not having label or notation
Warning Media studies is a Concept URI not having label or notation
Warning Theatre is a Concept URI not having label or notation
Warning consortium is a Concept URI not having label or notation
Warning Curricula is a Concept URI not having label or notation
Warning Research Process is a Concept URI not having label or notation
Warning DARIAH National Resource is a Concept URI not having label or notation
Warning Repositories & Collections is a Concept URI not having label or notation
Warning Concordancing/querying is a Concept URI not having label or notation
Warning Platform-independent (java) is a Concept URI not having label or notation
Warning SSHOC Key Exploitable Result is a Concept URI not having label or notation
Warning Consortium-HN is a Concept URI not having label or notation
Warning Concordancing/querying is a Concept URI not having label or notation
Warning Linux is a Concept URI not having la

/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

Warning DARIAH National Resource is a Concept URI not having label or notation
Warning DARIAH National Resource is a Concept URI not having label or notation
Warning Concordancing/querying is a Concept URI not having label or notation
Warning corpus building is a Concept URI not having label or notation
Warning Linux is a Concept URI not having label or notation
Warning MacOS is a Concept URI not having label or notation
Warning Windows is a Concept URI not having label or notation
Warning DARIAH National Resource is a Concept URI not having label or notation
Warning DARIAH National Resource is a Concept URI not having label or notation
Warning DARIAH National Resource is a Concept URI not having label or notation
Warning Concordancing/querying is a Concept URI not having label or notation
Warning Platform-independent (requires Python) is a Concept URI not having label or notation
Warning Annotating/concordancing/querying/listening to audio recordings is a Concept URI not having label 

/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

Warning Spoken corpora is a Concept URI not having label or notation
Warning phonemic is a Concept URI not having label or notation
Warning Spoken corpora is a Concept URI not having label or notation
Warning Dictionaries is a Concept URI not having label or notation
Warning orthographically transcribed is a Concept URI not having label or notation
Warning Spoken corpora is a Concept URI not having label or notation
Warning Turn segmentation is a Concept URI not having label or notation
Warning Spoken corpora is a Concept URI not having label or notation
Warning orthography is a Concept URI not having label or notation
Warning phonology is a Concept URI not having label or notation
Warning prosody is a Concept URI not having label or notation
Warning Multimodal corpora is a Concept URI not having label or notation
Warning speaker turn is a Concept URI not having label or notation
Warning noise is a Concept URI not having label or notation
Warning hand gesture is a Concept URI not havin

...

/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

Warning L2 learner corpora is a Concept URI not having label or notation
Warning L2 learner corpora is a Concept URI not having label or notation
Warning L2 learner corpora is a Concept URI not having label or notation
Warning L2 learner corpora is a Concept URI not having label or notation
Warning L2 learner corpora is a Concept URI not having label or notation
Warning Glossaries is a Concept URI not having label or notation
Warning Parallel corpora is a Concept URI not having label or notation
Warning audio/transcription linking is a Concept URI not having label or notation
Warning audio/transcription linking is a Concept URI not having label or notation
Warning L2 learner corpora is a Concept URI not having label or notation
Warning L2 learner corpora is a Concept URI not having label or notation
Warning audio/transcription linking is a Concept URI not having label or notation
Warning audio/transcription linking is a Concept URI not having label or notation
Warning L2 learner corpor

/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

Warning timealignment is a Concept URI not having label or notation
Warning English (mainly) is a Concept URI not having label or notation
Warning Oral History Corpora is a Concept URI not having label or notation
Warning verb senses (characterized by glosses and examples) is a Concept URI not having label or notation
Warning conjugation forms is a Concept URI not having label or notation
Warning phonetic forms is a Concept URI not having label or notation
Warning use frequencies is a Concept URI not having label or notation
Warning phonetic transcription is a Concept URI not having label or notation
Warning Multimodal corpora is a Concept URI not having label or notation
Warning videotranscription alignment is a Concept URI not having label or notation
Warning word segmentation is a Concept URI not having label or notation
Warning EAF transcripts is a Concept URI not having label or notation
Warning Sign language resources is a Concept URI not having label or notation
Warning Turkish 

/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  itemsnokw=itemsnokw.append(muit,  ignore_index=True)
/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_56455/4058194813.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

In [37]:
itemsnokw.head()

,MPUrl,label,code,code.uri
0,tool-or-service/OgfTLA,3DHOP: 3D Heritage Online Presenter,dariahNationalResource,https://vocabs.sshopencloud.eu/vocabularies/ss...
1,tool-or-service/lgyrt9,ACERR - The Arts and Culture in Education Rese...,Media+studies,https://vocabs.sshopencloud.eu/vocabularies/ss...
2,tool-or-service/lgyrt9,ACERR - The Arts and Culture in Education Rese...,Theatre,https://vocabs.sshopencloud.eu/vocabularies/ss...
3,tool-or-service/lgyrt9,ACERR - The Arts and Culture in Education Rese...,consortium,https://vocabs.sshopencloud.eu/vocabularies/ss...
4,tool-or-service/lgyrt9,ACERR - The Arts and Culture in Education Rese...,Curricula,https://vocabs.sshopencloud.eu/vocabularies/ss...
